In [1]:
import os
import requests
import json
import numpy as np
from typing import List, Dict, Any, Optional
from openai import OpenAI
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from dotenv import load_dotenv

In [2]:
from astrapy import DataAPIClient

In [3]:
load_dotenv()
openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [4]:
def connect_to_astradb():
    ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
    ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")

    client = DataAPIClient(token=ASTRA_DB_APPLICATION_TOKEN)
    database = client.get_database(ASTRA_DB_API_ENDPOINT)

    collection = database.get_collection("chat_radiology")
    return collection

In [5]:
def generate_embedding(text: str) -> List[float]:
    """Generate embedding for the given text using OpenAI's embedding model"""
    response = openai_client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [6]:
def search_documents(query: str, collection) -> List[Dict[str, Any]]:
    embedding = generate_embedding(query)
    cursor = collection.find(
        None,  
        sort={
            "$vector": embedding
        },
        limit=5
    )
    documents = list(cursor)
    
    return documents

In [8]:
def format_context(documents: List[Dict[str, Any]]) -> str:
    
    context = ""
    for doc in documents:
        filename = doc.get("category", "dokumen tidak diketahui")
        context += f"Sumber: {filename}\n{doc.get('text', '')}\n\n"
    return context

In [9]:
def create_rag_prompt(context: str, query: str) -> str:
    
    system_prompt = f"""
    Anda adalah asisten medis yang membantu menjawab pertanyaan berdasarkan informasi berikut:
    START CONTEXT
    {context}
    END CONTEXT

    Instruksi:
    1. Jawablah pertanyaan secara panjang, terstruktur, dan sangat mendetail.
    2. Apabila informasi berupa gejala-gejala yang dialami pasien, berikan lima kemungkinan diagnosis berdasarkan informasi tersebut, dengan penjelasan singkat yang menghubungkan gejala dengan setiap diagnosis. Kemudian, berikan saran kepada dokter untuk investigasi gejala lebih lanjut berdasarkan informasi dari dalam dokumen.
    3. Jika gejalanya menyangkut tentang nyeri atau sakit di ulu hati atau perut bagian atas, selalu berikan dispepsia sebagai salah satu diagnosis kemungkinan.
    4. Jika gejala yang disampaikan berhubungan dengan demam, seperti demam tinggi atau demam lebih dari 2 hari, selalu berikan demam biasa sebagai salah satu diagnosis kemungkinan, dan sertakan informasi tentang demam sedikit lebih detail seperti cara obat yang diperlukan.
    5. Jika gejala yang disampaikan berhubungan dengan nafas sesak atau batuk, kemungkinan besar pasien terkena penyakit asthma. Jadi, selalu berikan asthma sebagai salah satu diagnosis kemungkinan.
    6. Jika pertanyaan berkaitan dengan suatu penyakit (misalnya "Pasien terkena TB Paru. Jelaskan..."), berikan penjelasan selengkap dan sedetail mungkin tentang penyakit tersebut, mengacu secara langsung pada informasi yang ada dalam konteks (dokumen). Sertakan informasi tentang etiologi, gejala khas, diagnosa, penatalaksanaan, dan referensi dokumen. Jika tersedia, sebutkan nama file asal informasi tersebut.
    7. Apabila informasi bukan berupa gejala tetapi pertanyaan umum, jawablah secara lengkap dan rujuk sumber informasi dari dokumen dengan menyebutkan nama file.
    8. Jika Anda tidak yakin dengan jawaban (informasi tidak terdapat dalam konteks) atau pertanyaan kurang spesifik, katakan Anda tidak yakin dengan jawaban Anda.
    9. Jangan menambahkan informasi di luar konteks yang diberikan.
    10. Pastikan jawaban akurat, berdasarkan pedoman klinis resmi atau dokumen yang tersedia.
    11. Berikan peringatan bahwa ini bukan pengganti nasihat medis profesional dan pasien harus berkonsultasi dengan dokter untuk diagnosis dan perawatan yang akurat.
    12. Di akhir jawaban, sampaikan bahwa jawaban diambil berdasarkan Keputusan Menteri Kesehatan Republik Indonesia Nomor HK.01.07/MENKES/1186/2022 tentang Panduan Praktik Klinis bagi Dokter di Fasilitas Pelayanan Kesehatan Tingkat Pertama.
    """
    
    return f"{system_prompt}\n\n{query}"

In [10]:
def call_deepseek_api(prompt: str, model: str = "deepseek/deepseek-chat:free") -> str:
    
    url = "https://openrouter.ai/api/v1/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {os.environ.get('OPENROUTER_API_KEY')}",
        "HTTP-Referer": "https://radiologi.com",
        "X-Title": "Radiology GPT",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code != 200:
        raise Exception(f"Error calling DeepSeek API: {response.text}")
    
    return response.json()["choices"][0]["message"]["content"]

In [11]:
def process_with_rag(query: str, use_deepseek_r1: bool = False) -> str:
    model = "deepseek/deepseek-r1:free" if use_deepseek_r1 else "deepseek/deepseek-chat:free"
    collection = connect_to_astradb()
    documents = search_documents(query, collection)
    
    context = format_context(documents)
    prompt = create_rag_prompt(context, query)
    response = call_deepseek_api(prompt, model)
    
    return response

In [12]:
query = """Pasien mengalami demam pada waktu malam. Sebelum masuk rumah sakit, 
pasien merasa mual, muntah dan menggigil. Pasien didiagnosis penyakit apa?"""
    
response = process_with_rag(query, True)
print(response)

Berdasarkan gejala yang dialami pasien, berikut lima kemungkinan diagnosis beserta penjelasannya:

### 1. **Malaria**  
**Kesesuaian Gejala**:  
- Demam **hilang timbul** (termasuk demam malam hari).  
- Menggigil dan berkeringat saat demam turun (sumber: *Penyakit Umum, hal.40*).  
- Mual, muntah, nyeri otot, dan penurunan nafsu makan.  
- **Faktor risiko**: Tinggal atau bepergian ke daerah endemis malaria.  
**Pembeda Kunci**: Pola demam periodik (misalnya, demam setiap 24-72 jam tergantung spesies *Plasmodium*) dan riwayat eksposur daerah endemis.  

---

### 2. **Demam Tifoid (Tifoid Klinis Probabel)**  
**Kesesuaian Gejala**:  
- Demam tinggi yang mungkin lebih dominan pada malam hari.  
- Gejala gastrointestinal: mual, muntah, diare, atau sakit perut (sumber: *Digestive, hal.135*).  
- Bisa disertai gangguan kesadaran jika terjadi komplikasi (tifoid toksik).  
**Pembeda Kunci**: Demam progresif (meningkat bertahap), lidah kotor, dan bradikardi relatif.  

---

### 3. **Leptospiro

In [14]:
import requests
import json
import pandas as pd
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from bert_score import score

In [20]:
df = pd.read_excel('30_sample_penyakit.xlsx')
df = df.head(5)
df

,Kode,keluhan_utama,PARAPHRASE KELUHAN UTAMA,jalannya_penyakit,PARAPHRASE JALANNYA PENYAKIT,pemeriksaan_penunjang,hasil_laboratorium,diagnosa_utama,kode_icd_utama,diagnosa_sekunder_1,kode_icd_sekunder_1,kondisi_pulang,obat_waktu_pulang
0,A001,pasien datang dengan keluhan demam terutama pa...,Pasien mengalami demam pada waktu malam. Sebel...,sudah minum obat belum ada perubahan,Sudah minum obat dan ada perubahan.,"td: 120/80 hr: 100x t: 38,1",widal: (+) 1/160,Demam tifoid,A01.0,Tidak terdata,Tidak terdata,Hidup,floxifar 2x1 erlamol 3x1 metyl 2x1 rani 2x1
1,A002,bab cair lebih dari 6x sejak 1 hari lalu denga...,Pasien mengalami buang air besar cair lebih da...,- Pemeriksaan penunjang yang positif : TD 140/...,- Pemeriksaan penunjang yang positif : TD 140/...,"TD 140/70 N 88x R 21x S36,8 Kepala, thorax : d...",Tidak terdata,Other gastroenteritis and colitis of infectiou...,A09,Tidak terdata,Tidak terdata,inj ranitidin antasida 3x1 pct 31 lopamid prn,inj ranitidin antasida 3x1 pct 31 lopamid prn
2,A003,PASIEN DATANG DENGAN KELUHAN DEMAM 6 HARI SMRS...,Pasien datang dengan keluhan demam selama 6 ha...,SUDAH MINUM OBAT BELUM ADA PERUBAHAN Pemeriksa...,SUDAH MINUM OBAT BELUM ADA PERUBAHAN Pemeriksa...,"BB: 40KG T: 39,8 PTEKIE (+)",TR: 142RB,Dengue fever [classical dengue],A90,Tidak terdata,Tidak terdata,Hidup Obat-obatan waktu pulang/nasihat : PASIE...,PASIEN MENOLAK RAWAT INAP PCT 3X1 DEXTEEM 3X1 ...
3,B001,demam sejak 3 hari lalu disertai muncul bruntu...,Pasien menderita demam disertai munculnya brun...,sudah berobat namun beelum ada perbaikan Pemer...,Sudah berobat tapi belum ada perubahan. TD = 1...,TD 150/80 N 88x R 20x S 38 Status generalis db...,Tidak terdata,Zoster [herpes zoster],B02,"Fever, unspecified",R50.9,injeksi ketorolac pct 3x1 asiklovir 5x400mg vi...,injeksi ketorolac pct 3x1 asiklovir 5x400mg vi...
4,i001,pasien datang dengan keluhan nyeri dada sejak ...,Pasien nyeri dada sejak 4 jam lalu seperti ter...,nyeri berulang dengan durasi 20 menit TD : 125...,nyeri berulang dengan durasi 20 menit TD : 125...,tidak ada,Tidak terdata,hipertensi urgensi,i16,Tidak terdata,Tidak terdata,amlodipin 10 mg 1x1 captopril 25 mg 2x1 injeks...,amlodipin 10 mg 1x1 captopril 25 mg 2x1 injeks...


In [21]:
df["llm_response"] = df["PARAPHRASE KELUHAN UTAMA"].apply(process_with_rag)

In [22]:
cosine_similarities = []
bleu_scores = []
meteor_scores = []
bertP_scores = []
bertR_scores = []
bertF1_scores = []

In [23]:
for index, row in df.iterrows():
    ground_truth = str(row["diagnosa_utama"]).lower()  
    llm_output = str(row["llm_response"]).lower()  

    if not ground_truth or not llm_output:  
        cosine_similarities.append(-1)
        bertP_scores.append(-1)
        bertR_scores.append(-1)
        bertF1_scores.append(-1)
        bleu_scores.append(-1)
        meteor_scores.append(-1)
        continue

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([ground_truth, llm_output])
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    cosine_similarities.append(cosine_sim)

    bleu = sentence_bleu([ground_truth.split()], llm_output.split())
    bleu_scores.append(bleu)

    meteor = meteor_score([ground_truth.split()], llm_output.split())
    meteor_scores.append(meteor)

    P, R, F1 = score([llm_output], [ground_truth], lang="en", verbose=False)
    bertP_scores.append(P[0].item())
    bertR_scores.append(R[0].item())
    bertF1_scores.append(F1[0].item())

c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for p

In [24]:
df["cosine_similarity"] = cosine_similarities[:len(df)]
df["bleu_score"] = bleu_scores[:len(df)]
df["meteor_score"] = meteor_scores[:len(df)]
df["bert P"] = bertP_scores[:len(df)]
df["bert R"] = bertR_scores[:len(df)]
df["bert F1"] = bertF1_scores[:len(df)]

df[["PARAPHRASE KELUHAN UTAMA", "diagnosa_utama", "llm_response", 
    "bleu_score", "cosine_similarity", "meteor_score", "bert P", "bert R", "bert F1"]]

,PARAPHRASE KELUHAN UTAMA,diagnosa_utama,llm_response,bleu_score,cosine_similarity,meteor_score,bert P,bert R,bert F1
0,Pasien mengalami demam pada waktu malam. Sebel...,Demam tifoid,"Berdasarkan gejala yang dialami oleh pasien, y...",9.421538e-156,0.177537,0.050268,0.675021,0.825456,0.742697
1,Pasien mengalami buang air besar cair lebih da...,Other gastroenteritis and colitis of infectiou...,"Berdasarkan gejala yang dialami pasien, yaitu ...",4.043747e-232,0.007006,0.010142,0.670820,0.839298,0.745661
2,Pasien datang dengan keluhan demam selama 6 ha...,Dengue fever [classical dengue],Pasien datang dengan keluhan demam selama 6 ha...,4.014827e-232,0.055181,0.010870,0.667420,0.820070,0.735912
3,Pasien menderita demam disertai munculnya brun...,Zoster [herpes zoster],Berdasarkan gejala yang dialami pasien dan has...,0.000000e+00,0.000000,0.000000,0.665421,0.761295,0.710137
4,Pasien nyeri dada sejak 4 jam lalu seperti ter...,hipertensi urgensi,"Berdasarkan gejala yang dialami pasien, yaitu ...",0.000000e+00,0.008691,0.000000,0.688376,0.760367,0.722583


In [26]:
df.describe()

,cosine_similarity,bleu_score,meteor_score,bert P,bert R,bert F1
count,5.000000,5.000000e+00,5.000000,5.000000,5.000000,5.000000
mean,0.049683,1.884308e-156,0.014256,0.673412,0.801297,0.731398
std,0.074745,4.213440e-156,0.020807,0.009124,0.037602,0.014843
min,0.000000,0.000000e+00,0.000000,0.665421,0.760367,0.710137
25%,0.007006,0.000000e+00,0.000000,0.667420,0.761295,0.722583
50%,0.008691,4.014827e-232,0.010142,0.670820,0.820070,0.735912
75%,0.055181,4.043747e-232,0.010870,0.675021,0.825456,0.742697
max,0.177537,9.421538e-156,0.050268,0.688376,0.839298,0.745661
